# Velocidades propias

Obtener las velocidades propias de las parejas, para poder hacer los histogramas después.

In [1]:
import pandas as pd
import numpy as np

In [9]:
def del_ra(x,y):
    a = np.abs(x-y)
    b = np.abs(a-360.0)
    return np.minimum(a,b)

def cos_distang(ra1,dec1,ra2,dec2):
    dra  = np.deg2rad(del_ra(ra1,ra2))
    dec1 = np.deg2rad(dec1)
    dec2 = np.deg2rad(dec2)
    return np.sin(dec1)*np.sin(dec2) + np.cos(dec1)*np.cos(dec2)*np.cos(dra)

def dcd(ra1,dec1,ra2,dec2,dra1,ddec1,dra2,ddec2):
    dra    = np.abs(np.deg2rad(del_ra(ra1,ra2)))
    cd1sd2 = np.abs(np.cos(np.deg2rad(dec1))*np.sin(np.deg2rad(dec2)))
    sd1cd2 = np.abs(np.sin(np.deg2rad(dec1))*np.cos(np.deg2rad(dec2)))
    dra1   = np.deg2rad(dra1)
    dra2   = np.deg2rad(dra2)
    ddec1  = np.deg2rad(ddec1)
    ddec2  = np.deg2rad(ddec2)
    
    return cd1sd2*ddec1 + sd1cd2*ddec2 + np.cos(dra)*(
    sd1cd2*ddec1 + cd1sd2*ddec2) + np.cos(np.deg2rad(dec1))*np.cos(np.deg2rad(dec2))*np.sin(dra)*(dra1+dra2)

def R3D(r1,r2,cost):
    """
    Separación tridimensional
    """
    return np.sqrt((r2-r1)**2 + 2.*r1*r2*(1.-cost))

def dR3D(r1,r2,cost,dr1,dr2,dcost):
    """
    Error en la separación tridimensional
    """
    return (np.abs(r2-r1)*(dr1+dr2) + r2*r1*dcost + (1.-cost)*(r1*dr2 + r2*dr1))/R3D(r1,r2,cost)

def vel_tan_diff(pmra_1,pmdec_1,dist_1,pmra_2,pmdec_2,dist_2):
    """Tangential velocity difference using pm in ra and dec directions, and distances in pc,
    in km/s
    Only valid for small angles.
    """
    return np.sqrt((pm2kms(pmra_2,dist_2)-pm2kms(pmra_1,dist_1))**2 +
                   (pm2kms(pmdec_2,dist_2)-pm2kms(pmdec_1,dist_1))**2)

def vel_tan_diff_err(pmra_1,pmra_1_err,pmdec_1,pmdec_1_err,dist_1,dist_1_err,
                     pmra_2,pmra_2_err,pmdec_2,pmdec_2_err,dist_2,dist_2_err):
    """Tangential velocity difference error using pm in ra and dec directions, and distances in pc,
    in km/s
    Only valid for small angles
    """
    diffvra  = np.abs((pm2kms(pmra_2,dist_2)-pm2kms(pmra_1,dist_1))*
                      (pm2kms_error(pmra_1,pmra_1_err,dist_1,dist_1_err)+pm2kms_error(pmra_2,pmra_2_err,dist_2,dist_2_err)))
    diffvdec = np.abs((pm2kms(pmdec_2,dist_2)-pm2kms(pmdec_1,dist_1))*
                      (pm2kms_error(pmdec_1,pmdec_1_err,dist_1,dist_1_err)+pm2kms_error(pmdec_2,pmdec_2_err,dist_2,dist_2_err)))
    return (diffvra+diffvdec)/vel_tan_diff(pmra_1,pmdec_1,dist_1,pmra_2,pmdec_2,dist_2)

def pm2kms(pm,dist):
    """Proper motions in mas/yr to km/s, using distance in pc"""
    return (np.pi/(180*3600*1000))*(1./(365*24*3600))*(3.086E13)*pm*dist
def pm2kms_error(pm,pm_err,dist,dist_err):
    """Proper motions error in mas/yr to km/s, using distance in pc"""
    return pm2kms(pm,dist)*(pm_err/pm + dist_err/dist)

In [3]:
# Archivo donde está la lista preprocesada
picklefile='../data_processed/jimenez_on_gaia_pairs_only.pkl'

#importar la tabla
tab = pd.read_pickle(picklefile)

In [11]:
# Definir los valores para GDR2

tab['distance_x']           = 1000./tab.parallax_x
tab['distance_error_x']     = tab.distance_x*tab.parallax_error_x/tab.parallax_x
tab['distance_y']           = 1000./tab.parallax_y
tab['distance_error_y']     = tab.distance_y*tab.parallax_error_y/tab.parallax_y

tab['dpmra_GDR2']         = tab.pmra_y - tab.pmra_x
tab['dpmra_GDR2_err']     = tab.pmra_error_y + tab.pmra_error_x
tab['dpmde_GDR2']         = tab.pmdec_y - tab.pmdec_x
tab['dpmde_GDR2_err']     = tab.pmdec_error_y + tab.pmdec_error_x

tab['del_pmra_GDR2']      = tab.pmra_y - tab.pmra_x
#point['del_pmra']        = -point.pmra_x*np.abs(1-point.distance_y/point.distance_x) + \
#    (-point.pmra_y*np.sin(np.deg2rad(point.dec_y)))*del_ra(point.ra_x,point.ra_y)*np.pi/180.
tab['del_pmra_GDR2_err']  = tab.pmra_error_y + tab.pmra_error_x
tab['del_pmde_GDR2']      = tab.pmdec_y - tab.pmdec_x
#point['del_pmdec']       =  -point.pmdec_x*np.abs(1-point.distance_y/point.distance_x) + \
#    (point.pmdec_y*np.sin(np.deg2rad(point.dec_y)))*del_ra(point.ra_x,point.ra_y)*np.pi/180.
tab['del_pmde_GDR2_err']  = tab.pmdec_error_y + tab.pmdec_error_x

#usando la fórmula de distancia angular, ley de cosenos

tab['cos_angsep_GDR2']      = cos_distang(tab.ra_x, tab.dec_x,
                                          tab.ra_y, tab.dec_y)

tab['dcos_angsep_GDR2']     = dcd(tab.ra_x, tab.dec_x, tab.ra_y, tab.dec_y,
                                  tab.ra_error_x/3.6E6, tab.dec_error_x/3.6E6,
                                  tab.ra_error_y/3.6E6, tab.dec_error_y/3.6E6)

tab['ang_sep_GDR2']     = np.rad2deg(np.arccos(tab.cos_angsep_GDR2))

tab['ang_sep_GDR2_err'] = np.rad2deg(tab.dcos_angsep_GDR2 / np.sqrt(1.-tab.cos_angsep_GDR2**2))

tab['proy_sep_GDR2']        = R3D(500*(1./tab.parallax_x + 1./tab.parallax_y),500*(1./tab.parallax_x + 1./tab.parallax_y),
                              tab.cos_angsep_GDR2)
tab['proy_sep_error_GDR2']  = dR3D(500*(1./tab.parallax_x + 1./tab.parallax_y),500*(1./tab.parallax_x + 1./tab.parallax_y),
                                tab.cos_angsep_GDR2,
                                1000*(tab.parallax_error_x/tab.parallax_x**2 + tab.parallax_error_y/tab.parallax_y**2),
                                1000*(tab.parallax_error_x/tab.parallax_x**2 + tab.parallax_error_y/tab.parallax_y**2),
                                tab.dcos_angsep_GDR2)

# Relative velocity on the sky perpendicular component
tab['rel_vel_GDR2']       = vel_tan_diff(tab.pmra_x,tab.pmdec_x,1000./tab.parallax_x,
                                         tab.pmra_y,tab.pmdec_y,1000./tab.parallax_y)

tab['rel_vel_GDR2_err'] = vel_tan_diff_err(tab.pmra_x,tab.pmra_error_x,tab.pmdec_x,tab.pmdec_error_x,
                                             1000./tab.parallax_x,1000.*tab.parallax_error_x/(tab.parallax_x)**2,
                                             tab.pmra_y,tab.pmra_error_y,tab.pmdec_y,tab.pmdec_error_y,
                                             1000./tab.parallax_y,1000.*tab.parallax_error_y/(tab.parallax_y)**2)

#Use Jimenez correction
if (True):
    tab.pmra_x  = tab.pmra_corrected_x
    tab.pmra_y  = tab.pmra_corrected_y
    tab.pmdec_x = tab.pmdec_corrected_x
    tab.pmdec_y = tab.pmdec_corrected_y
    
    tab.pmra_error_x  = tab.pmra_corrected_error_x
    tab.pmra_error_y  = tab.pmra_corrected_error_y
    tab.pmdec_error_x = tab.pmdec_corrected_error_x
    tab.pmdec_error_y = tab.pmdec_corrected_error_y

# Efectos de proyeccion
k = 4.74
tab['U'] = -(k*tab.pmra_x/tab.parallax_x)*np.sin(np.deg2rad(tab.ra_x)) \
           -(k*tab.pmdec_x/tab.parallax_x)*np.cos(np.deg2rad(tab.ra_x))*np.sin(np.deg2rad(tab.dec_x)) \
           +tab.radial_velocity_x.fillna(0)*np.cos(np.deg2rad(tab.ra_x))*np.cos(np.deg2rad(tab.dec_x))

tab['V'] =  (k*tab.pmra_x/tab.parallax_x)*np.cos(np.deg2rad(tab.ra_x)) \
           -(k*tab.pmdec_x/tab.parallax_x)*np.sin(np.deg2rad(tab.ra_x))*np.sin(np.deg2rad(tab.dec_x)) \
           +tab.radial_velocity_x.fillna(0)*np.sin(np.deg2rad(tab.ra_x))*np.cos(np.deg2rad(tab.dec_x))

tab['W'] =  (k*tab.pmdec_x/tab.parallax_x)*np.cos(np.deg2rad(tab.dec_x)) \
            +tab.radial_velocity_x.fillna(0)*np.sin(np.deg2rad(tab.dec_x))

tab['pmra_corr'] = tab.pmra_x - (-tab.U*np.sin(np.deg2rad(tab.ra_y)) +
                    tab.V*np.cos(np.deg2rad(tab.ra_y)))*tab.parallax_y/k

tab['pmde_corr'] = tab.pmdec_x - (-tab.U*np.cos(np.deg2rad(tab.ra_y))*np.sin(np.deg2rad(tab.dec_y)) -
                    tab.V*np.sin(np.deg2rad(tab.ra_y))*np.sin(np.deg2rad(tab.dec_y)) +
                    tab.W*np.cos(np.deg2rad(tab.dec_y)))*tab.parallax_y/k

tab['rvel_corr'] = tab.radial_velocity_x-(tab.U*np.cos(np.deg2rad(tab.ra_y))*np.cos(np.deg2rad(tab.dec_y)) + \
                    tab.V*np.sin(np.deg2rad(tab.ra_y))*np.cos(np.deg2rad(tab.dec_y)) + \
                    tab.W*np.sin(np.deg2rad(tab.dec_y)))

tab['dvelra_corr'] = tab.pmra_y*k/tab.parallax_y - (tab.pmra_x - tab.pmra_corr)*k/tab.parallax_x

tab['dvelde_corr'] = tab.pmdec_y*k/tab.parallax_y - (tab.pmdec_x - tab.pmde_corr)*k/tab.parallax_x
    
#============================================================================================
#============================================================================================

k = 4.74
tab['U1'] = -(k*tab.pmra_y/tab.parallax_y)*np.sin(np.deg2rad(tab.ra_y)) \
           -(k*tab.pmdec_y/tab.parallax_y)*np.cos(np.deg2rad(tab.ra_y))*np.sin(np.deg2rad(tab.dec_y)) \
           +tab.radial_velocity_y.fillna(0)*np.cos(np.deg2rad(tab.ra_y))*np.cos(np.deg2rad(tab.dec_y))

tab['V1'] =  (k*tab.pmra_y/tab.parallax_y)*np.cos(np.deg2rad(tab.ra_y)) \
           -(k*tab.pmdec_y/tab.parallax_y)*np.sin(np.deg2rad(tab.ra_y))*np.sin(np.deg2rad(tab.dec_y)) \
           +tab.radial_velocity_y.fillna(0)*np.sin(np.deg2rad(tab.ra_y))*np.cos(np.deg2rad(tab.dec_y))

tab['W1'] =  (k*tab.pmdec_y/tab.parallax_y)*np.cos(np.deg2rad(tab.dec_y)) \
            +tab.radial_velocity_y.fillna(0)*np.sin(np.deg2rad(tab.dec_y))

tab['pmra_corr1'] = tab.pmra_y - (-tab.U1*np.sin(np.deg2rad(tab.ra_x)) +
                    tab.V1*np.cos(np.deg2rad(tab.ra_x)))*tab.parallax_x/k

tab['pmde_corr1'] = tab.pmdec_y - (-tab.U1*np.cos(np.deg2rad(tab.ra_x))*np.sin(np.deg2rad(tab.dec_x)) -
                    tab.V1*np.sin(np.deg2rad(tab.ra_x))*np.sin(np.deg2rad(tab.dec_x)) +
                    tab.W1*np.cos(np.deg2rad(tab.dec_x)))*tab.parallax_x/k
    
tab['rvel_corr1'] = tab.radial_velocity_y-(tab.U1*np.cos(np.deg2rad(tab.ra_x))*np.cos(np.deg2rad(tab.dec_x)) + \
                    tab.V1*np.sin(np.deg2rad(tab.ra_x))*np.cos(np.deg2rad(tab.dec_x)) + \
                    tab.W1*np.sin(np.deg2rad(tab.dec_x)))

#============================================================================================
#============================================================================================

tab['U2'] = -(k*tab.pmra_x/tab.parallax_x)*np.sin(np.deg2rad(tab.ra_x)) \
           -(k*tab.pmdec_x/tab.parallax_x)*np.cos(np.deg2rad(tab.ra_x))*np.sin(np.deg2rad(tab.dec_x)) 

tab['V2'] =  (k*tab.pmra_x/tab.parallax_x)*np.cos(np.deg2rad(tab.ra_x)) \
           -(k*tab.pmdec_x/tab.parallax_x)*np.sin(np.deg2rad(tab.ra_x))*np.sin(np.deg2rad(tab.dec_x))

tab['W2'] =  (k*tab.pmdec_x/tab.parallax_x)*np.cos(np.deg2rad(tab.dec_x)) 

tab['pmra_corr2'] = tab.pmra_x - (-tab.U2*np.sin(np.deg2rad(tab.ra_y)) +
                    tab.V2*np.cos(np.deg2rad(tab.ra_y)))*tab.parallax_y/k

tab['pmde_corr2'] = tab.pmdec_x - (-tab.U2*np.cos(np.deg2rad(tab.ra_y))*np.sin(np.deg2rad(tab.dec_y)) -
                    tab.V2*np.sin(np.deg2rad(tab.ra_y))*np.sin(np.deg2rad(tab.dec_y)) +
                    tab.W2*np.cos(np.deg2rad(tab.dec_y)))*tab.parallax_y/k
    
tab['rvel_corr2'] = tab.radial_velocity_x-(tab.U2*np.cos(np.deg2rad(tab.ra_y))*np.cos(np.deg2rad(tab.dec_y)) + \
                    tab.V2*np.sin(np.deg2rad(tab.ra_y))*np.cos(np.deg2rad(tab.dec_y)) + \
                    tab.W2*np.sin(np.deg2rad(tab.dec_y)))

tab['rv_consist_GDR2'] = np.abs((tab.radial_velocity_y/tab.radial_velocity_x-1.)/
                                (tab.radial_velocity_error_x/tab.radial_velocity_x +
                                 tab.radial_velocity_error_y/tab.radial_velocity_y))

tab['rv_consist_RAVE'] = np.abs((tab.radial_velocity_y/tab.radial_velocity_x-1.)/
                                (tab.radial_velocity_error_x/tab.radial_velocity_x +
                                 tab.radial_velocity_error_y/tab.radial_velocity_y))

In [13]:
#Seleccion de calidad:
sigma = 2.5
tab = tab[(~(tab.radial_velocity_x.isna() | tab.radial_velocity_y.isna()) &
    (tab.rv_consist_GDR2 < sigma) &
    (tab.radial_velocity_error_x < 5) &
    (tab.radial_velocity_error_y < 5))|
    (~(tab.hrv_x.isna() | tab.hrv_y.isna()) &
    (tab.rv_consist_RAVE < sigma) &
    (tab.e_hrv_x < 5) &
    (tab.e_hrv_y < 5))
   ]

- Con GDR2, encuentran 3741 candidatos a binarias abiertas (pair AND multiple systems)
- Usan la mag límite de Ty2 = 11.5 para M completo a 40 pc
                                   G completo a 200 pc
                                   F completo a 400 pc
- Con errores < 10% en p.m. y parallax
- Con separaciones < 500,000 AU
- Igual paralaje y p.m. en r.a. y dec (dentro de 2.5$\sigma$ => 0.13 mas; 0.2 mas/yr)
- Sin corrección geométrica.
    Esto da 3852 candidatos (dobles + múltiples), 11 834 estrellas
- Gaia DR2 + RAVE da vr para 3318 de las estrellas.
- Con errores < 5 km/s, se reduce a 2877 de Gaia DR-2 y 282 de RAVE
- Sólo para 678 pares había RV para ambas componentes.
- 16% (108) fueron discrepantes en RV, 66 de los múltiples también
- En total, 570 pares con RV concordantes

In [14]:
tab.to_pickle('../data_processed/je_prop_vel_sel_je.pkl')